In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install pyperclip

In [ ]:
driver = webdriver.Chrome('/content/chromedriver.exe')

##사용할 변수 선언
#네이버 로그인 주소
url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
uid = '아이디'
upw = '패스워드!'

#네이버 로그인 페이지로 이동
driver.get(url)
time.sleep(2) #로딩 대기

#아이디 입력폼
tag_id = driver.find_element_by_name('id')
#패스워드 입력폼
tag_pw = driver.find_element_by_name('pw')

# id 입력
# 입력폼 클릭 -> paperclip에 선언한 uid 내용 복사 -> 붙여넣기
tag_id.click()
pyperclip.copy(uid)
tag_id.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

# pw 입력
# 입력폼 클릭 -> paperclip에 선언한 upw 내용 복사 -> 붙여넣기
tag_pw.click()
pyperclip.copy(upw)
tag_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

#로그인 버튼 클릭
login_btn = driver.find_element_by_id('log.login')
login_btn.click()
time.sleep(2)

In [76]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pyperclip

In [1]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import time
import numpy

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
#액션장르
#초반에 성인 웹툰이 있어서 테스트할 때 쓰세요
#성인 웹툰 진입은 안돼요
#성인 웹툰이 없는 스포츠 카테고리 같은 건 실행 다 됩니다
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append(["제목","완결여부", "평점", "작가", "업데이트 년도", "형식", "장르", "이용가", "별점참여수", "요약"])

raw = requests.get("https://comic.naver.com/webtoon/genre?view=list&order=User&genre=action",
                   headers = {"User-Agent" : "Mozilla/5.0"})
html = BeautifulSoup(raw.text, 'html.parser')
webtoons = html.select("tbody > tr")

for webtoon in webtoons:
    title = webtoon.select_one("td:nth-of-type(1) a")
    url = title.attrs["href"]
    
    ends = webtoon.select_one("td.subject > span")

    score = webtoon.select_one("div.rating_type > strong").text
    author = webtoon.select_one("td:nth-of-type(3) a").text

    date = webtoon.select_one("td.date").text
    date = date.replace('\t', '')
    date = date.replace('\n', '')
    date = date[0:4]

    each_raw = requests.get("https://comic.naver.com" + url,
                            headers = {"User-Agent" : "Mozilla/5.0"})
    each_html = BeautifulSoup(each_raw.text, 'html.parser')

    genre = each_html.select_one("p.detail_info span.genre").text
    genre = genre.split(', ')

    storysum = each_html.select_one("div.detail p").text
    age = each_html.select_one("span.age").text


    urls = each_html.select("td > a")
    avg_p =[]

    for i in urls:
        each_ep = i.attrs["href"]
        if each_ep[0] == 'h':
          continue
        epsode = requests.get("https://comic.naver.com" + each_ep,
                            headers = {"User-Agent" : "Mozilla/5.0"})
        ep_html =  BeautifulSoup(epsode.text, 'html.parser')
        ep = ep_html.select_one("span.pointTotalPerson > em").text
        ep = int(ep)
        avg_p.append(ep)
         

    avg = numpy.mean(avg_p)

    print("제목 :"+title.text)
    print("-"*20)
    if ends is not None:
        print("완결여부 : 완결함")
    else:
        print("완결여부 : 연재중")
    print("평점 :"+ score)
    print("작가 :"+ author)
    print("업데이트 년도 :"+ date)
    print("형식 :"+ genre[0])
    print("장르 :"+ genre[1])
    print("이용가 :"+ age)
    print("별점참여수 : " +str(avg))
    print("요약 :"+storysum)
    print("="*50)

    sheet.append([title.text, float(score), author, date, genre[0], genre[1], age, avg, storysum])


제목 :김부장
--------------------
완결여부 : 연재중
평점 :9.27
작가 :박태준 만화회사 / 정종택
업데이트 년도 :2022
형식 :스토리
장르 :액션
이용가 :15세 이용가
별점참여수 : 14135.3
요약 :“제발 안경 쓴 아저씨는 건들지 말자…”오직 자신의 딸 '민지'를 위해 특수요원직을 관두고 평범함을 선택한 가장 김부장.그러던 어느 날 민지가 소리소문 없이 사라지고, 김부장은 자신을 감시하는 국가를 적으로 돌리면서까지 딸을 찾아나서기 시작하는데...[외모지상주의], [싸움독학] 그리고 [인생존망]의 세계관을 잇는 공식 스핀오프 작품!
제목 :참교육
--------------------
완결여부 : 연재중
평점 :9.86
작가 :채용택 / 한가람
업데이트 년도 :2022
형식 :스토리
장르 :액션
이용가 :15세 이용가
별점참여수 : 20368.6
요약 :무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!<부활남> 채용택 작가 X <신석기녀> 한가람 작가의 신작!
제목 :싸움독학
--------------------
완결여부 : 연재중
평점 :9.71
작가 :박태준 만화회사 / 김정현...
업데이트 년도 :2022
형식 :스토리
장르 :액션
이용가 :15세 이용가
별점참여수 : 17309.7
요약 :힘없고 가진거 하나 없이 맞고만 살던 나였는데...우연히 비밀의 뉴튜브를 발견하게 되고 갑자기 떼돈을 벌었다.
제목 :약한영웅
--------------------
완결여부 : 연재중
평점 :9.82
작가 :서패스 / 김진석
업데이트 년도 :2022
형식 :에피소드
장르 :액션
이용가 :15세 이용가
별점참여수 : 10924.4
요약 :“비겁하다…? 애초에 동등한 시작이 아닌데... 체급도 쪽수도 안 맞잖아- 이 X끼들아!!”선천적으로 약한 소년이 상대보다 몇 수 앞을 예측하는 심리전과지형지물을 이용하고, 도구로 살벌하게 끝장내는 파이터로 성장한다.이제 그의 또 다른 이름은 '은장백사'다.


AttributeError: ignored

In [98]:
from ast import keyword
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append(["제목", "평점", "작가", "업데이트 년도", "형식", "장르", "이용가", "별점참여수", "요약", "조회순"])

raw = requests.get("https://comic.naver.com/webtoon/finish?view=list&order=ViewCount",
                   headers = {"User-Agent" : "Mozilla/5.0"})
html = BeautifulSoup(raw.text, 'html.parser')
webtoons = html.select("tbody > tr")

count = 1
for webtoon in webtoons:
  title = webtoon.select_one("td:nth-of-type(1) a")
  url = title.attrs["href"]
    
  ends = webtoon.select_one("td.subject > span")

  score = webtoon.select_one("div.rating_type > strong").text
  author = webtoon.select_one("td:nth-of-type(3) a").text

  date = webtoon.select_one("td.date").text
  date = date.replace('\t', '')
  date = date.replace('\n', '')
  date = date[0:4]

  each_raw = requests.get("https://comic.naver.com" + url,
                            headers = {"User-Agent" : "Mozilla/5.0"})
  each_html = BeautifulSoup(each_raw.text, 'html.parser')

  genre = each_html.select_one("p.detail_info span.genre").text
  genre = genre.split(', ')

  storysum = each_html.select_one("div.detail p").text
  age = each_html.select_one("span.age")
  if age is None:
    age = "전체연령가"
  else:
    age = age.text

    urls = each_html.select("td > a")
    avg_p =[]
    temp = 'start'
    
  
  if age =="18세 이용가"
  #로그인 연동
    for i in urls:
        each_ep = i.attrs["href"]
        if each_ep[0] != '/' or each_ep == temp:
          continue
        print(each_ep)
        epsode = requests.get("https://comic.naver.com" + each_ep, jar=j,
                            headers = {"User-Agent" : "Mozilla/5.0"})
        ep_html =  BeautifulSoup(epsode.text, 'html.parser')
        ep = ep_html.select_one("span.pointTotalPerson > em").text
        ep = int(ep)
        avg_p.append(ep)
        temp = each_ep
  else:
    for i in urls:
        each_ep = i.attrs["href"]
        if each_ep[0] != '/' or each_ep == temp:
          continue
        print(each_ep)
        epsode = requests.get("https://comic.naver.com" + each_ep,
                            headers = {"User-Agent" : "Mozilla/5.0"})
        ep_html =  BeautifulSoup(epsode.text, 'html.parser')
        ep = ep_html.select_one("span.pointTotalPerson > em").text
        ep = int(ep)
        avg_p.append(ep)
        temp = each_ep

  print("제목 :"+title.text)
  print("-"*20)
  print("평점 :"+ score)
  print("작가 :"+ author)
  print("업데이트 년도 :"+ date)
  print("형식 :"+ genre[0])
  print("장르 :"+ genre[1])
  print("이용가 :"+ age)
  print("별점참여수 : " +str(avg))
  print("요약 :"+storysum)
  print("조회순 :"+str(count))
  print("="*50)

  sheet.append([title.text, float(score), author, date, genre[0], genre[1], age, avg, storysum, count])
  count += 1

wb.save("NaverToonInfo.xlsx")

SyntaxError: ignored